# Test Lien Site Web - App Kivy

Compilé avec google colab pour créer le .apk.   
App lancé dans Android.

## Installation sur l'envt google colab avec toutes les dépendances

In [ ]:
!apt-get update -qq
!apt-get install -y \
    openjdk-17-jdk \
    zip unzip \
    autoconf \
    libtool \
    pkg-config \
    zlib1g-dev \
    libncurses5-dev \
    libncursesw5-dev \
    libtinfo5 \
    cmake \
    libffi-dev \
    libssl-dev \
    automake
!pip install -q buildozer==1.5.0 cython==0.29.33 kivy
!mkdir -p /content/myapp
%cd /content/myapp

## Création des fichiers nécessaires

1. main.py pour créer l'app
2. buildozer.spec pour la compiler
3. manifest.xml pour détecter l'appel par un site web

### Python - main.py

In [ ]:
%%writefile main.py
from kivy.app import App
from kivy.uix.label import Label
from kivy.uix.button import Button
from kivy.uix.boxlayout import BoxLayout
from kivy.utils import platform
from kivy.clock import Clock
from urllib.parse import urlparse, parse_qs
import requests

def get_android_launch_url():
    try:
        from jnius import autoclass

        PythonActivity = autoclass('org.kivy.android.PythonActivity')
        activity = PythonActivity.mActivity

        intent = activity.getIntent()
        action = intent.getAction()

        if action == "android.intent.action.VIEW":
            uri = intent.getData()
            if uri:
                return uri.toString()
    except Exception as e:
        print("Erreur Intent:", e)

    return None


def send_params(id_, key):
    url = f"https://www.planete-sciences.org/espace/spock/api.html?object=stabtraj&id={id_}&key={key}"

    DF = {
        "object": "stabtraj",
        "id": id_,
        "key": key,
        "data": {
            "Long_ogive": 120
        }
    }

    try:
        r = requests.put(url, json=DF)
        print("Envoi terminé :", r.status_code)
    except Exception as e:
        print("Erreur request :", e)


def open_website(url):
    try:
        from jnius import autoclass, cast
        Intent = autoclass('android.content.Intent')
        Uri = autoclass('android.net.Uri')
        PythonActivity = autoclass('org.kivy.android.PythonActivity')

        intent = Intent(Intent.ACTION_VIEW)
        intent.setData(Uri.parse(url))

        current_activity = PythonActivity.mActivity
        current_activity.startActivity(intent)
    except Exception as e:
        print("Erreur ouverture site :", e)

class MyApp(App):

    def build(self):
        # Layout vertical
        self.layout = BoxLayout(orientation='vertical', padding=20, spacing=20)

        # Label
        self.label = Label(text="---- Bienvenue ----\n(En attente de lien...)")
        self.layout.add_widget(self.label)

        # Bouton (caché au début)
        self.button = Button(text="Envoyer à SPOCK", size_hint=(1, 0.3))
        self.button.disabled = True
        self.button.opacity = 0
        self.button.bind(on_press=self.on_button_press)

        self.layout.add_widget(self.button)

         # Nouveau bouton
        self.button_site = Button(text="Ouvrir le site", size_hint=(1, 0.3))
        self.button_site.disabled = True
        self.button_site.opacity = 0
        self.button_site.bind(on_press=self.on_site_press)

        self.layout.add_widget(self.button_site)

        return self.layout


    def on_start(self):
        if platform == "android":
            Clock.schedule_once(self.read_link, 0.6)


    def read_link(self, dt):
        link = get_android_launch_url()

        if link:
            parsed = urlparse(link)

            params = parse_qs(parsed.query)
            self.data = params.get("data", [""])[0]
            self.id_  = params.get("id", [""])[0]
            self.key  = params.get("key", [""])[0]
            self.code  = params.get("code", [""])[0]

            call = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"

            self.label.text = (
                f"---- Bienvenue ----\n"
                f"Appel par : {call}\n"
                f"id : {self.id_}\n"
                f"key : {self.key}"
            )

            # Activer bouton
            self.button.disabled = False
            self.button.opacity = 1
            # Activer bouton
            self.button_site.disabled = False
            self.button_site.opacity = 1

        else:
            self.label.text = "---- Bienvenue ----\nAucun lien reçu"


    def on_button_press(self, instance):
        if hasattr(self, "id_") and hasattr(self, "key"):
            send_params(self.id_, self.key)
            self.label.text += "\n\nParamètres envoyés ! (ogive = 120)"
        else:
            self.label.text += "\n\nImpossible : pas de paramètres."

    def on_site_press(self, instance):
        open_website(f"https://www.planete-sciences.org/espace/spock/stabtraj.html?code={self.code}")

if __name__ == "__main__":
    MyApp().run()

### Buildozer - spec

In [ ]:
%%writefile buildozer.spec
[app]
title = MyApp
package.name = myapp
package.domain = org.test
source.dir = .
source.include_exts = py,png,jpg,kv
version = 0.1
requirements = python3,kivy,jnius,android
orientation = portrait

# Permissions Android
android.permissions = INTERNET

# API et NDK
android.api = 31
android.minapi = 21
android.ndk = 25b
android.archs = arm64-v8a

# Fichier manifest supplémentaire pour deep linking
android.manifest.intent_filters = manifest_pmi.xml

[buildozer]
log_level = 2
warn_on_root = 0

### Manifest - xml

In [ ]:
%%writefile manifest_pmi.xml

android.manifest.intent_filters = <intent-filter>
    <action android:name="android.intent.action.VIEW"/>
    <category android:name="android.intent.category.DEFAULT"/>
    <category android:name="android.intent.category.BROWSABLE"/>
    <data android:scheme="monapp" android:host="ouvrir"/>
</intent-filter>

## Compilation de l'app

In [ ]:
!buildozer -v android debug

## Téléchargement depuis google colab

In [ ]:
import os
from google.colab import files

# Chercher l'APK
!find . -name "*.apk" -type f

# Liste tous les APK trouvés
apk_files = []
for root, dirs, files_list in os.walk('.'):
    for file in files_list:
        if file.endswith('.apk'):
            apk_files.append(os.path.join(root, file))

if apk_files:
    print(f"APK trouvé : {apk_files[0]}")
    files.download(apk_files[0])
else:
    print("Aucun APK trouvé. Vérifier les erreurs ci-dessus.")
    # Afficher les dernières erreurs
    !tail -50 .buildozer/android/logs/*.log 2>/dev/null